# Social Determinants of Health Database Dashboard

[Source](https://www.ahrq.gov/sdoh/data-analytics/sdoh-data.html)

In [78]:
#Import Libraries Needed
import pandas as pd
import geopandas as gpd
import panel as pn
pn.extension('plotly',comms='ipywidgets')
import param
from bokeh.resources import INLINE
import hvplot.pandas
import plotly.express as px
from urllib.request import urlopen
import plotly.graph_objects as go
import requests
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import seaborn as sns

In [58]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
css = '''
.bk.panel-widget-box {
  background: #f0f0f0;
  border-radius: 5px;
  border: 1px black solid;
  font-size: 24px;
}
'''

pn.extension(raw_css=[css])

In [4]:
df = pd.read_csv('../data/SDOH/sdoh_2020_tract_1_0.csv').drop('YEAR', axis=1)
df

,TRACTFIPS,COUNTYFIPS,STATEFIPS,STATE,COUNTY,REGION,TERRITORY,ACS_TOT_POP_WT,ACS_TOT_POP_US_ABOVE1,ACS_TOT_POP_ABOVE5,...,WUSTL_AVG_PM25,HIFLD_DIST_UC_TRACT,POS_DIST_ED_TRACT,POS_DIST_MEDSURG_ICU_TRACT,POS_DIST_TRAUMA_TRACT,POS_DIST_PED_ICU_TRACT,POS_DIST_OBSTETRICS_TRACT,POS_DIST_CLINIC_TRACT,POS_DIST_ALC_TRACT,CEN_AIAN_NH_IND
0,1001020100,1001,1,Alabama,Autauga County,South,0,1941.0,1928.0,1843.0,...,6.79,3.52,2.25,2.25,12.72,55.04,12.72,10.30,13.11,0
1,1001020200,1001,1,Alabama,Autauga County,South,0,1757.0,1753.0,1651.0,...,7.15,2.64,1.36,1.36,11.89,55.65,11.89,10.99,12.24,0
2,1001020300,1001,1,Alabama,Autauga County,South,0,3694.0,3672.0,3586.0,...,7.12,1.99,0.85,0.85,11.48,55.75,11.48,10.98,11.71,0
3,1001020400,1001,1,Alabama,Autauga County,South,0,3539.0,3519.0,3409.0,...,7.30,0.97,0.55,0.55,10.55,56.49,10.55,9.95,10.70,0
4,1001020501,1001,1,Alabama,Autauga County,South,0,4306.0,4158.0,3912.0,...,7.58,0.69,1.79,1.79,8.95,57.99,8.95,8.55,9.13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85523,78030960900,78030,78,US Virgin Islands,St. Thomas Island,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
85524,78030961000,78030,78,US Virgin Islands,St. Thomas Island,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
85525,78030961100,78030,78,US Virgin Islands,St. Thomas Island,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
85526,78030961200,78030,78,US Virgin Islands,St. Thomas Island,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [16]:
# Count unique values in each column
unique_counts = df.nunique()

# Calculate min, max, and median
min_count = unique_counts.min()
max_count = unique_counts.max()
median_count = unique_counts.median()

print(f"Minimum unique count: {min_count}")
print(f"Maximum unique count: {max_count}")
print(f"Median unique count: {median_count}")

Minimum unique count: 2
Maximum unique count: 85528
Median unique count: 4442.0


In [61]:
print(df.columns.tolist())

['TRACTFIPS', 'COUNTYFIPS', 'STATEFIPS', 'STATE', 'COUNTY', 'REGION', 'TERRITORY', 'ACS_TOT_POP_WT', 'ACS_TOT_POP_US_ABOVE1', 'ACS_TOT_POP_ABOVE5', 'ACS_TOT_POP_ABOVE15', 'ACS_TOT_POP_ABOVE16', 'ACS_TOT_POP_16_19', 'ACS_TOT_POP_ABOVE25', 'ACS_TOT_CIVIL_POP_ABOVE18', 'ACS_TOT_CIVIL_VET_POP_ABOVE25', 'ACS_TOT_OWN_CHILD_BELOW17', 'ACS_TOT_CIVILIAN_LABOR', 'ACS_TOT_CIVIL_EMPLOY_POP', 'ACS_TOT_WORKER_HH', 'ACS_TOT_WORKER_NWFH', 'ACS_TOT_CIVIL_NONINST_POP_POV', 'ACS_TOT_CIVIL_POP_POV', 'ACS_TOT_POP_POV', 'ACS_TOT_GRANDCHILDREN_GP', 'ACS_TOT_HU', 'ACS_TOT_HH', 'ACS_AVG_HH_SIZE', 'ACS_TOT_CIVIL_NONINST_POP', 'ACS_TOT_CIVIL_VET_POP', 'ACS_PCT_CHILD_DISAB', 'ACS_PCT_DISABLE', 'ACS_PCT_NONVET_DISABLE_18_64', 'ACS_PCT_VET_DISABLE_18_64', 'ACS_PCT_FEMALE', 'ACS_PCT_MALE', 'ACS_PCT_CTZ_ABOVE18', 'ACS_PCT_CTZ_NATURALIZED', 'ACS_PCT_CTZ_NONUS_BORN', 'ACS_PCT_CTZ_US_BORN', 'ACS_PCT_FOREIGN_BORN', 'ACS_PCT_NON_CITIZEN', 'ACS_PCT_NONCTN_1990', 'ACS_PCT_NONCTN_1999', 'ACS_PCT_NONCTN_2000', 'ACS_PCT_NONCTN

In [36]:
dash_df = df.copy()

In [37]:
# Get unique sorted values for COUNTY and STATE
countynames = sorted(dash_df['COUNTY'].unique())
fips = sorted(dash_df['STATE'].unique())

# Group by STATE and calculate mean
dash_df = dash_df.groupby('STATE').mean().reset_index()

# Assign COUNTY and STATE back to the dataframe
dash_df['COUNTY'] = dash_df['STATE'].map(dict(zip(fips, countynames)))

dash_df

,STATE,TRACTFIPS,COUNTYFIPS,STATEFIPS,TERRITORY,ACS_TOT_POP_WT,ACS_TOT_POP_US_ABOVE1,ACS_TOT_POP_ABOVE5,ACS_TOT_POP_ABOVE15,ACS_TOT_POP_ABOVE16,ACS_TOT_POP_16_19,ACS_TOT_POP_ABOVE25,ACS_TOT_CIVIL_POP_ABOVE18,ACS_TOT_CIVIL_VET_POP_ABOVE25,ACS_TOT_OWN_CHILD_BELOW17,ACS_TOT_CIVILIAN_LABOR,ACS_TOT_CIVIL_EMPLOY_POP,ACS_TOT_WORKER_HH,ACS_TOT_WORKER_NWFH,ACS_TOT_CIVIL_NONINST_POP_POV,ACS_TOT_CIVIL_POP_POV,ACS_TOT_POP_POV,ACS_TOT_GRANDCHILDREN_GP,ACS_TOT_HU,ACS_TOT_HH,ACS_AVG_HH_SIZE,ACS_TOT_CIVIL_NONINST_POP,ACS_TOT_CIVIL_VET_POP,ACS_PCT_CHILD_DISAB,ACS_PCT_DISABLE,ACS_PCT_NONVET_DISABLE_18_64,ACS_PCT_VET_DISABLE_18_64,ACS_PCT_FEMALE,ACS_PCT_MALE,ACS_PCT_CTZ_ABOVE18,ACS_PCT_CTZ_NATURALIZED,ACS_PCT_CTZ_NONUS_BORN,ACS_PCT_CTZ_US_BORN,ACS_PCT_FOREIGN_BORN,ACS_PCT_NON_CITIZEN,ACS_PCT_NONCTN_1990,ACS_PCT_NONCTN_1999,ACS_PCT_NONCTN_2000,ACS_PCT_NONCTN_2010,ACS_PCT_API_LANG,ACS_PCT_ENGL_NOT_ALL,ACS_PCT_ENGL_NOT_WELL,ACS_PCT_ENGL_VERY_WELL,ACS_PCT_ENGL_WELL,ACS_PCT_ENGLISH,ACS_PCT_HH_LIMIT_ENGLISH,ACS_PCT_OTH_EURP,ACS_PCT_OTH_LANG,ACS_PCT_SPANISH,ACS_PCT_GULFWAR_1990_2001,ACS_PCT_GULFWAR_2001,ACS_PCT_GULFWAR_VIETNAM,ACS_PCT_VET,ACS_PCT_VIETNAM,ACS_MEDIAN_AGE,ACS_MEDIAN_AGE_FEMALE,ACS_MEDIAN_AGE_MALE,ACS_PCT_AGE_0_4,ACS_PCT_AGE_5_9,ACS_PCT_AGE_10_14,ACS_PCT_AGE_15_17,ACS_PCT_AGE_0_17,ACS_PCT_AGE_18_29,ACS_PCT_AGE_18_44,ACS_PCT_AGE_30_44,ACS_PCT_AGE_45_64,ACS_PCT_AGE_50_64,ACS_PCT_AGE_ABOVE65,ACS_PCT_AGE_ABOVE80,ACS_PCT_AIAN,ACS_PCT_AIAN_FEMALE,ACS_PCT_AIAN_MALE,ACS_PCT_AIAN_NONHISP,ACS_PCT_ASIAN,ACS_PCT_ASIAN_FEMALE,ACS_PCT_ASIAN_MALE,ACS_PCT_ASIAN_NONHISP,ACS_PCT_BLACK,ACS_PCT_BLACK_FEMALE,ACS_PCT_BLACK_MALE,ACS_PCT_BLACK_NONHISP,ACS_PCT_HISP_FEMALE,ACS_PCT_HISP_MALE,ACS_PCT_HISPANIC,ACS_PCT_MULT_RACE,ACS_PCT_MULT_RACE_FEMALE,ACS_PCT_MULT_RACE_MALE,ACS_PCT_MULT_RACE_NONHISP,ACS_PCT_NHPI,ACS_PCT_NHPI_FEMALE,ACS_PCT_NHPI_MALE,ACS_PCT_NHPI_NONHISP,ACS_PCT_OTHER_FEMALE,ACS_PCT_OTHER_MALE,ACS_PCT_OTHER_NONHISP,ACS_PCT_OTHER_RACE,ACS_PCT_WHITE,ACS_PCT_WHITE_FEMALE,ACS_PCT_WHITE_MALE,ACS_PCT_WHITE_NONHISP,ACS_PCT_HOUSEHOLDER_WHITE,ACS_PCT_HOUSEHOLDER_BLACK,ACS_PCT_HOUSEHOLDER_AIAN,ACS_PCT_HOUSEHOLDER_ASIAN,ACS_PCT_HOUSEHOLDER_NHPI,ACS_PCT_HOUSEHOLDER_OTHER,ACS_PCT_HOUSEHOLDER_MULT,ACS_PCT_AIAN_COMB,ACS_PCT_ASIAN_COMB,ACS_PCT_BLACK_COMB,ACS_PCT_NHPI_COMB,ACS_PCT_WHITE_COMB,ACS_PCT_HH_NO_COMP_DEV,ACS_PCT_HH_SMARTPHONE,ACS_PCT_HH_SMARTPHONE_ONLY,ACS_PCT_HH_TABLET,ACS_PCT_HH_TABLET_ONLY,ACS_PCT_HH_PC,ACS_PCT_HH_PC_ONLY,ACS_PCT_HH_OTHER_COMP,ACS_PCT_HH_OTHER_COMP_ONLY,ACS_PCT_HH_INTERNET,ACS_PCT_HH_INTERNET_NO_SUBS,ACS_PCT_HH_BROADBAND,ACS_PCT_HH_BROADBAND_ONLY,ACS_PCT_HH_BROADBAND_ANY,ACS_PCT_HH_CELLULAR,ACS_PCT_HH_CELLULAR_ONLY,ACS_PCT_HH_NO_INTERNET,ACS_PCT_HH_SAT_INTERNET,ACS_PCT_HH_DIAL_INTERNET_ONLY,ACS_PCT_CHILD_1FAM,ACS_PCT_CHILDREN_GRANDPARENT,ACS_PCT_GRANDP_RESPS_NO_P,ACS_PCT_GRANDP_RESPS_P,ACS_PCT_GRANDP_NO_RESPS,ACS_PCT_HH_KID_1PRNT,ACS_PCT_DIVORCED_F,ACS_PCT_DIVORCED_M,ACS_PCT_MARRIED_SP_AB_F,ACS_PCT_MARRIED_SP_AB_M,ACS_PCT_MARRIED_SP_PR_F,ACS_PCT_MARRIED_SP_PR_M,ACS_PCT_NVR_MARRIED_F,ACS_PCT_NVR_MARRIED_M,ACS_PCT_WIDOWED_F,ACS_PCT_WIDOWED_M,ACS_PCT_POP_SAME_SEX_UNMRD_P,ACS_PCT_POP_SAME_SEX_SPOUSE,ACS_PCT_ADMIN,ACS_PCT_ARMED_FORCES,ACS_PCT_ART,ACS_PCT_CONSTRUCT,ACS_PCT_EDUC,ACS_PCT_EMPLOYED,ACS_PCT_FINANCE,ACS_PCT_GOVT,ACS_PCT_INFORM,ACS_PCT_MANUFACT,ACS_PCT_NATURE,ACS_PCT_NOT_LABOR,ACS_PCT_OTHER,ACS_PCT_PROFESS,ACS_PCT_PVT_NONPROFIT,ACS_PCT_PVT_PROFIT,ACS_PCT_RETAIL,ACS_PCT_TRANSPORT,ACS_PCT_UNEMPLOY,ACS_PCT_VET_LABOR_FORCE_18_64,ACS_PCT_VET_UNEMPL_18_64,ACS_PCT_WHOLESALE,ACS_PCT_WORK_RES_F,ACS_PCT_WORK_RES_M,ACS_GINI_INDEX,ACS_MDN_GRNDPRNT_NO_PRNT_INC,ACS_MDN_GRNDPRNT_INC,ACS_MEDIAN_HH_INC_AIAN,ACS_MEDIAN_HH_INC_ASIAN,ACS_MEDIAN_HH_INC_BLACK,ACS_MEDIAN_HH_INC_HISP,ACS_MEDIAN_HH_INC_MULTI,ACS_MEDIAN_HH_INC_NHPI,ACS_MEDIAN_HH_INC_OTHER,ACS_MEDIAN_HH_INC_WHITE,ACS_MEDIAN_HH_INC,ACS_MEDIAN_INC_F,ACS_MEDIAN_INC_M,ACS_MEDIAN_NONVET_INC,ACS_MEDIAN_VET_INC,ACS_PCT_INC50_ABOVE65,ACS_PCT_INC50_BELOW17,ACS_PCT_HEALTH_INC_BELOW137,ACS_PCT_HEALTH_INC_

In [42]:
dash_df = dash_df.fillna(0).astype({col: 'int64' for col in dash_df.select_dtypes('float64').columns})
print(dash_df.dtypes)

STATE                            object
TRACTFIPS                         int64
COUNTYFIPS                        int64
STATEFIPS                         int64
TERRITORY                         int64
ACS_TOT_POP_WT                    int64
ACS_TOT_POP_US_ABOVE1             int64
ACS_TOT_POP_ABOVE5                int64
ACS_TOT_POP_ABOVE15               int64
ACS_TOT_POP_ABOVE16               int64
ACS_TOT_POP_16_19                 int64
ACS_TOT_POP_ABOVE25               int64
ACS_TOT_CIVIL_POP_ABOVE18         int64
ACS_TOT_CIVIL_VET_POP_ABOVE25     int64
ACS_TOT_OWN_CHILD_BELOW17         int64
ACS_TOT_CIVILIAN_LABOR            int64
ACS_TOT_CIVIL_EMPLOY_POP          int64
ACS_TOT_WORKER_HH                 int64
ACS_TOT_WORKER_NWFH               int64
ACS_TOT_CIVIL_NONINST_POP_POV     int64
ACS_TOT_CIVIL_POP_POV             int64
ACS_TOT_POP_POV                   int64
ACS_TOT_GRANDCHILDREN_GP          int64
ACS_TOT_HU                        int64
ACS_TOT_HH                        int64


In [44]:
dash_df.columns.tolist()

['STATE',
 'TRACTFIPS',
 'COUNTYFIPS',
 'STATEFIPS',
 'TERRITORY',
 'ACS_TOT_POP_WT',
 'ACS_TOT_POP_US_ABOVE1',
 'ACS_TOT_POP_ABOVE5',
 'ACS_TOT_POP_ABOVE15',
 'ACS_TOT_POP_ABOVE16',
 'ACS_TOT_POP_16_19',
 'ACS_TOT_POP_ABOVE25',
 'ACS_TOT_CIVIL_POP_ABOVE18',
 'ACS_TOT_CIVIL_VET_POP_ABOVE25',
 'ACS_TOT_OWN_CHILD_BELOW17',
 'ACS_TOT_CIVILIAN_LABOR',
 'ACS_TOT_CIVIL_EMPLOY_POP',
 'ACS_TOT_WORKER_HH',
 'ACS_TOT_WORKER_NWFH',
 'ACS_TOT_CIVIL_NONINST_POP_POV',
 'ACS_TOT_CIVIL_POP_POV',
 'ACS_TOT_POP_POV',
 'ACS_TOT_GRANDCHILDREN_GP',
 'ACS_TOT_HU',
 'ACS_TOT_HH',
 'ACS_AVG_HH_SIZE',
 'ACS_TOT_CIVIL_NONINST_POP',
 'ACS_TOT_CIVIL_VET_POP',
 'ACS_PCT_CHILD_DISAB',
 'ACS_PCT_DISABLE',
 'ACS_PCT_NONVET_DISABLE_18_64',
 'ACS_PCT_VET_DISABLE_18_64',
 'ACS_PCT_FEMALE',
 'ACS_PCT_MALE',
 'ACS_PCT_CTZ_ABOVE18',
 'ACS_PCT_CTZ_NATURALIZED',
 'ACS_PCT_CTZ_NONUS_BORN',
 'ACS_PCT_CTZ_US_BORN',
 'ACS_PCT_FOREIGN_BORN',
 'ACS_PCT_NON_CITIZEN',
 'ACS_PCT_NONCTN_1990',
 'ACS_PCT_NONCTN_1999',
 'ACS_PCT_NONCTN_

In [43]:
dash_df.to_csv('../data/SDOH/CountyLevelstats.csv')

In [63]:
county_dict = pd.DataFrame()
county_dict['County #'] = dash_df['COUNTYFIPS']
county_dict['County Name'] = dash_df['COUNTY']
cdict = pd.Series(county_dict['County #'].values,index=county_dict['County Name']).to_dict()
ndict = pd.Series(county_dict['County Name'].values,index=county_dict['County #']).to_dict()
#label_data['County Number'] = label_data['County #']
#label_data.replace({'County #': ndict}, inplace=True)
#label_data.rename(columns = {'County #': 'County Name','County Number':'County #'}, inplace=True)
cdict

{'Abbeville County': 1075,
 'Acadia Parish': 2098,
 'Accomack County': 60031,
 'Ada County': 4015,
 'Adair County': 5080,
 'Adams County': 6055,
 'Addison County': 8046,
 'Adjuntas Municipio': 9005,
 'Aguada Municipio': 10003,
 'Aguadilla Municipio': 11001,
 'Aguas Buenas Municipio': 12071,
 'Aibonito Municipio': 13130,
 'Aiken County': 66010,
 'Aitkin County': 15003,
 'Alachua County': 16029,
 'Alamance County': 17079,
 'Alameda County': 18089,
 'Alamosa County': 19110,
 'Albany County': 20119,
 'Albemarle County': 21112,
 'Alcona County': 22060,
 'Alcorn County': 23014,
 'Aleutians East Borough': 24087,
 'Aleutians West Census Area': 25017,
 'Alexander County': 26107,
 'Alexandria city': 27079,
 'Alfalfa County': 28075,
 'Alger County': 29141,
 'Allamakee County': 30054,
 'Allegan County': 31088,
 'Allegany County': 32018,
 'Alleghany County': 33011,
 'Allegheny County': 34019,
 'Allen County': 35023,
 'Allen Parish': 36062,
 'Allendale County': 37103,
 'Alpena County': 38050,
 'Alpi

In [64]:
county_dict.to_csv('countyfipsmatcher.csv')

# Texas County Level Map

In [54]:
import json

input_list = ['FIPS',
              'Population',
              'Demographics',
              'Income',
              'Housing']
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
target_states = ['48']
counties['features'] = [f for f in counties['features'] if f['properties']['STATE'] in target_states]

In [55]:
class MapDashboard(param.Parameterized):
    prefix_groups = {
        'FIPS': ['TRACTFIPS', 'COUNTYFIPS', 'STATEFIPS'],
        'Population': [col for col in dash_df.columns if 'POP' in col],
        'Demographics': [col for col in dash_df.columns if 'PCT' in col],
        'Income': [col for col in dash_df.columns if 'INC' in col],
        'Housing': [col for col in dash_df.columns if 'HU' in col or 'RENT' in col or 'HOME' in col]
    }

    # List of column names to be used in the selector
    input_list = [col for group in prefix_groups.values() for col in group]

    # Param selector to choose a variable dynamically
    _ = param.Selector(input_list, default=input_list[0])

    def get_map(self):
        # Select the chosen column from dash_df based on the selection from the param.Selector
        data = dash_df[['COUNTY', 'STATE', self._]]  # self._ refers to the selected column

        # Create the map using Plotly
        fig = px.choropleth_mapbox(data, geojson=counties, locations='STATE', color=self._,
            color_continuous_scale='balance_r',
            range_color=(-10, 10),
            center={'lat': 31.9686, 'lon': -99.9018},
            zoom=4.65,
            height=585, width=525,
            mapbox_style="white-bg",
            custom_data=[data['COUNTY'], self._]
        )

        hovertemp = '%{customdata[0]} County<br>%{customdata[1]}% change in ' + self._
        fig.update_traces(hovertemplate=hovertemp)
        fig.update_geos(fitbounds='locations')
        fig.update_layout(
            hoverlabel=dict(bgcolor='black', font_size=24, font_family="Times New Roman"),
            font=dict(family='times new roman', size=24, color='black'),
            margin={"r": 10, "t": 0, "l": 10, "b": 0},
            coloraxis_colorbar=dict(tickvals=[-8, -4, 0, 4, 8], ticktext=['-10', '-5', '0', '5', '10']),
        )
        fig.update_coloraxes(cmid=0, colorbar_orientation='h', colorbar_y=-0.148, colorbar_title_font_family='Times New Roman',
                             colorbar_title_font_size=20, colorbar_title_side='bottom', colorbar_thickness=4)
        return fig

    def get_description(self):
        """Provide a description based on the selected prefix group."""
        descriptions = {
            'FIPS': 'Contains geographic identifiers, including census tract, county, and state FIPS codes.',
            'Population': 'Includes population-related statistics, such as total population or population by group.',
            'Demographics': 'Represents demographic percentages, such as racial or age distributions.',
            'Income': 'Includes income-related metrics like median household income or income inequality.',
            'Housing': 'Represents housing-related data, including homeownership rates, housing units, and rent statistics.',
        }

        # Determine the prefix group based on the selected variable
        selected_prefix = None
        for prefix, columns in self.prefix_groups.items():
            if self._ in columns:
                selected_prefix = prefix
                break

        # Return the corresponding description
        description_text = descriptions.get(selected_prefix, 'No description available for the selected variable.')
        return pn.pane.HTML(description_text,
                            style={'color': 'black', 'font-size': '20px', 'width': '125%',
                                   'text-align': 'left', 'font-family': 'Times New Roman'})

    def panel(self):
        return pn.Row(
            pn.Column(
                pn.pane.HTML("""<h1>Select A Variable To View At The County Level</h1>""",
                              style={'color': 'black', 'font-size': '14px', 'width': '100%',
                                     'text-align': 'center',
                                     'font-family': 'Times New Roman'}),
                pn.Param(self, width=550, background='#FFFFFF',
                         widgets={'_': {'widget_type': pn.widgets.RadioButtonGroup}}),
                pn.Spacer(height=50),
                pn.pane.HTML("""<h1>What does this variable show us?</h1>""",
                            style={'color': 'black', 'font-size': '14px', 'width': '100%',
                                   'text-align': 'center',
                                   'font-family': 'Times New Roman'}),
                pn.Spacer(height=5),
                self.get_description
            ),
            pn.Spacer(width=150),
            self.get_map
        )


# Create the dashboard
dashboard = MapDashboard(name='')
map_component1 = dashboard.panel()
map_component1.save('CountyLevel_Tab', resources=INLINE)


# Heatmap & Histogram

In [ ]:
grouped_df = df.groupby(['COUNTY', 'STATE'])

# Select the columns for heatmap
columns_of_interest = ['ACS_TOT_POP_WT', 'ACS_TOT_POP_US_ABOVE1', 'ACS_TOT_POP_ABOVE5', 'ACS_TOT_POP_ABOVE15', 'ACS_TOT_POP_16_19']

# Calculate correlation matrix for selected columns
correlation_matrix = grouped_df[columns_of_interest].corr()

# Create a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Correlation Heatmap for Selected Columns")
plt.show()

In [ ]:
for column in columns_of_interest:
    plt.figure(figsize=(10, 6))
    grouped_df[column].plot(kind='hist', bins=20, alpha=0.7)
    plt.title(f'Histogram of {column} by County-State')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
# Combine both histograms and heatmaps in a subplot
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', ax=axes[0])
axes[0].set_title("Correlation Heatmap")

grouped_df['ACS_TOT_POP_WT'].plot(kind='hist', bins=20, alpha=0.7, ax=axes[1])
axes[1].set_title('Histogram of ACS_TOT_POP_WT')
axes[1].set_xlabel('ACS_TOT_POP_WT')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# County Level Dashboard

In [72]:
#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Social Determinants of Health Database Census Tract Data 2020</h1>""",
                              style={'color':'white','font-size':'20px','width':'145%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})

datalab_img = pn.pane.PNG("C:/Users/Mirna Elizondo/OneDrive - Texas State University/Desktop/Research/PAIR/data/datalabimage.png", height=171, width=280)
txst_img = pn.pane.PNG("C:/Users/Mirna Elizondo/OneDrive - Texas State University/Desktop/Research/PAIR/data/txstlog.png",height=140,width=350)


title_bar = pn.Row(pn.Spacer(width=30),datalab_img,pn.Spacer(width=55),title_html_pane,pn.Spacer(width=310),txst_img)

content_bar = pn.Row(pn.Spacer(width=225),map_component1)

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=6, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
gspec[1:6, 7] = content_bar
gspec.show()

Launching server at http://localhost:49344


# Heatmap & Histogram Dashboard

In [ ]:
#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Social Determinants of Health Database Census Tract Data 2020</h1>""",
                              style={'color':'white','font-size':'20px','width':'145%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})
datalab_img = pn.pane.PNG("C:/Users/Mirna Elizondo/OneDrive - Texas State University/Desktop/Research/PAIR/data/datalabimage.png", height=171, width=280)
txst_img = pn.pane.PNG("C:/Users/Mirna Elizondo/OneDrive - Texas State University/Desktop/Research/PAIR/data/txstlog.png",height=140,width=350)



title_bar = pn.Row(pn.Spacer(width=30),datalab_img,pn.Spacer(width=55),title_html_pane,pn.Spacer(width=310),txst_img)

content_bar = pn.Row(pn.Spacer(width=100),loss_component1, pn.Spacer(width=80),loss_component2)

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=6, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
gspec[2:6, 7] = content_bar
gspec.show()

# Definitions

In [ ]:


#Panel HTML elements can take a 'style' parameter directly. 
#You could alternatively assign a css class as above and include the css code in raw_css
title_html_pane = pn.pane.HTML("""<h1>Social Determinants of Health Database Census Tract Data 2020</h1>""",
                              style={'color':'white','font-size':'20px','width':'145%',
                                     'background-color':'#6A5638','text-align':'center',
                                     'outline-style':'solid','outline-color':'#30231D',
                                     'outline-width':'thick','font-family':'Times New Roman'})
datalab_img = pn.pane.PNG("C:/Users/Mirna Elizondo/OneDrive - Texas State University/Desktop/Research/PAIR/data/datalabimage.png", height=171, width=280)
txst_img = pn.pane.PNG("C:/Users/Mirna Elizondo/OneDrive - Texas State University/Desktop/Research/PAIR/data/txstlog.png",height=140,width=350)



title_bar = pn.Row(pn.Spacer(width=30),datalab_img,pn.Spacer(width=55),title_html_pane,pn.Spacer(width=310),txst_img)
index_pane = pn.pane.HTML("""<h1>Variable Definitions</h1>""", 
                          style={'color':'white','width':'168%','font-size':'24px','text-align':'center',    
                      #    'outline-style':'solid','outline-color':'#30231D',   
                      #    'outline-width':'thick',
                                 'background-color':'#501214',
                          'font-family':'Times New Roman',
                          'border':'10px solid black'})

        descriptions = {
            'FIPS': 'Contains geographic identifiers, including census tract, county, and state FIPS codes.',
            'Population': 'Includes population-related statistics, such as total population or population by group.',
            'Demographics': 'Represents demographic percentages, such as racial or age distributions.',
            'Income': 'Includes income-related metrics like median household income or income inequality.',
            'Housing': 'Represents housing-related data, including homeownership rates, housing units, and rent statistics.',
        }
    
table = pn.pane.HTML(
"""
<table style="width:105%">
<th style = 'font-size':'8px'>
  <tr>
    <td>FIPS</td>
    <td>Contains geographic identifiers, including census tract, county, and state FIPS codes</td>
  </tr>
  <tr>
    <td>Population</td>
    <td>Includes population-related statistics, such as total population or population by group</td>
  </tr>
  <tr>
    <td>Demographics</td>
    <td>Represents demographic percentages, such as racial or age distributions.</td>
 </tr>
 <tr>
    <td>Income</td>
    <td>Includes income-related metrics like median household income or income inequality.</td>
 </tr>
 <tr>
    <td>Housing</td>
    <td>Represents housing-related data, including homeownership rates, housing units, and rent statistics.</td>
 </tr>
</table>
""", style={'width':'168%','font-size':'32px','text-align':'center',    
            #'outline-style':'solid','outline-color':'#30231D',   
           # 'outline-width':'thick',
            'font-family':'Times New Roman',
            'border':'4px solid black'})
content_bar = pn.Row(pn.Spacer(width=145),
                     pn.Column(index_pane,table))

#Create a new gridspec with 14 columns and 12 rows
gspec = pn.GridSpec(ncols=14,nrows=12, background='#FFFFFF',sizing_mode='stretch_both')
#Place application elements in the grid, using pn.Spacer() for improved layout spacing and control
gspec[0, 7] = title_bar
#gspec[1:6, 0] = pn.Spacer()
#gspec[1:6, 5] = pn.Spacer()
gspec[2:6, 7] = content_bar
gspec.show()

